# 環境の確認とデータセットの読み出し

以降では、Pythonの機械学習用ライブラリーであるscikit-learnを利用するため、
バージョンを確認しておきます。

0.21以上であれば、以降のプログラムを実行することが可能なはずです。

また、著者当てゲームの特徴抽出済みのデータセットを読み出します。

In [1]:
 # ライブラリーのインポート
import sklearn
print(sklearn.__name__, sklearn.__version__)
import numpy as np
print(np.__name__, np.__version__)
import matplotlib.pyplot as plt
import japanize_matplotlib
import math
import random
import pandas as pd

# データセットの読み出し
akutagawa_name = np.load('data/akutagawa_name.npy', allow_pickle=True)
kikuchi_name = np.load('data/kikuchi_name.npy', allow_pickle=True)
hgram = np.load('data/hgram.npy', allow_pickle=True)
words = np.load('data/words.npy', allow_pickle=True)
print('\n芥川龍之介の作品は以下の20編')
print(' '.join(akutagawa_name))
print('\n菊池寛の作品は以下の20編')
print(' '.join(kikuchi_name))
print('\n40編の作品に現れる語の総数は{}語'.format(len(words)))

sklearn 1.0.2
numpy 1.22.1

芥川龍之介の作品は以下の20編
羅生門 妖婆 藪の中 貉 鼻 歯車 トロッコ 杜子春 俊寛 侏儒の言葉 邪宗門 将軍 死後 アグニの神 或る日の大石内蔵助 おぎん お時儀 河童 煙管 蜘蛛の糸

菊池寛の作品は以下の20編
姉川の戦い ある恋の話 入れ札 M公爵と写真師 屋上の狂人 恩讐の彼方に 女強盗 恩を返す話 義民甚平 勲章を貰う話 極楽 出世 勝負事 大力の物語 藤十郎の恋 ある抗議書 身投げ救助業 無名作家の日記 若杉裁判長 奉行と人相学

40編の作品に現れる語の総数は16307語


### 2.7.3 混同行列（confusion matrix）と評価指標
著者当てゲームの40編の作品データに対して、
5分割交差検証とk-NNアルゴリズム（$k = 1$）により予測を行います。

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter("ignore", FutureWarning)


def dl2(dictx, dicty):
    sq = 0
    for key in set(dictx.keys()) | set(dicty.keys()):
        x, y = 0, 0
        if key in dictx: 
            x = dictx[key]
        if key in dicty:
            y = dicty[key]
        sq += (x - y)**2
    return math.sqrt(sq)

temp = [] 
for dictx in hgram:
    temp.append(list(map(lambda dicty: dl2(dictx, dicty), hgram)))
dl2_matrix = np.array(temp) # 距離の行列
l1 = len(akutagawa_name)
l2 = len(kikuchi_name)
labels = np.array([0]*l1 + [1]*l2)

# ランダムに5群に分割
import random
name = np.hstack([akutagawa_name, kikuchi_name]) # 40作品すべてのタイトルのリスト生成

a_index = list(range(20)) # 芥川の作品は20(番号と作品が結びつけたい)
random.shuffle(a_index) # ランダムにシャッフル
a_partition = [a_index[4*n:4*n+4] for n in range(5)] # 芥川の20作品を四つずつ、5グループに分けてリストにする
k_index = list(range(20, 40))
random.shuffle(k_index)
k_partition = [k_index[4*n:4*n+4] for n in range(5)]
partition = [a_partition[n] + k_partition[n] for n in range(5)] # 前半四つが芥川、後半四つが菊池の作品番号を持つリストを合計五つ持つリスト
y_test = [0]*4 + [1]*4

clf = KNeighborsClassifier(metric='precomputed', n_neighbors=1)

prediction = []

y_test = [0]*4 + [1]*4
y_train = y_test*4

for i in range(5):
    index_train = sum([partition[n] for n in range(5) if n != i], [])
    dl2_train = dl2_matrix[index_train][:, index_train]
    dl2_test = dl2_matrix[partition[i]][:, index_train]

    clf.fit(dl2_train, y_train)
    prediction.append(clf.predict(dl2_test))

dict = {}
dict['グループ'] = np.array([[i]*8 for i in range(5)]).flatten()
dict['正解'] = ['芥川' if t == 0 else '菊池' for t in y_test*5]
dict['予測'] = ['']*40
df = pd.DataFrame(dict, index = [name[n] for n in np.array(partition).flatten()])

for i in range(5):
    for j in range(8):
        df.at[name[partition[i][j]], '予測'] = '芥川' if prediction[i][j] == 0 else '菊池'
display(df)


,グループ,正解,予測
妖婆,0,芥川,芥川
お時儀,0,芥川,芥川
アグニの神,0,芥川,菊池
将軍,0,芥川,芥川
姉川の戦い,0,菊池,菊池
大力の物語,0,菊池,菊池
恩讐の彼方に,0,菊池,菊池
若杉裁判長,0,菊池,菊池
藪の中,1,芥川,菊池
或る日の大石内蔵助,1,芥川,芥川


学習器を評価する最も基本的な指標は「正解率」ですが、より学習器の性格を見るためには他の指標にも注目する必要があります。今回最終的にハイパーパラメータ$k$の値を決めるために用いる指標は「正解率」ですが、ここではその他の指標についても紹介します。

始めに、学習器のアルゴリズムの成績を評価する上で重要となる混同行列について説明します。

**混同行列（Confusion Matrix）**を作成しましょう。
> - 列は予測
> - 行は正解

上のセルの予測結果を集計して、下の表の空欄に当てはまる数を見つけて下さい。

|  | 予測が芥川 | 予測が菊池 |
| :---- | :----: | :----: |
| **実際は芥川** |  |  |
| **実際は菊池** | | |

次のセルを実行して、答え合わせをしてみて下さい。

In [5]:
 # Confusion Matrixの作成
tp = len(df.query("正解=='芥川' & 予測=='芥川'"))
fp = len(df.query("正解=='菊池' & 予測=='芥川'"))
fn = len(df.query("正解=='芥川' & 予測=='菊池'"))
tn = len(df.query("正解=='菊池' & 予測=='菊池'"))
conf_matrix = pd.DataFrame([[tp, fn], [fp, tn]])
conf_matrix.index = ['実際は芥川', '実際は菊池']
conf_matrix.columns = ['芥川と予測', '菊池と予測']
display(conf_matrix)
acc = (tp+tn)/(tp+fp+fn+tn)

,芥川と予測,菊池と予測
実際は芥川,15,5
実際は菊池,6,14


正解率 = (15 + 14)/(15 + 6 + 5 + 14) = 0.725


混同行列から正解率は次のように計算されます。

In [ ]:
print('正解率 = ({0} + {3})/({0} + {1} + {2} + {3}) = {4}'.format(tp, fp, fn, tn, acc))

一般に混同行列のクラスは「陽性」「陰性」と表します。

>歴史的に、疾病検査薬の有効性の測定問題が、
二値分類問題の典型的な例として取り上げられてきたことから、
二値問題におけるクラスを
陽性（Positive）・陰性（Negative）とすることがあります。
検査薬は、特定の疾病への罹患を判定するための試薬であり、
疾病を直接に確認するものではないため、
検査薬による判定結果はあくまでも予測でしかありません。
疾病に罹患している場合を陽性、
罹患していない場合を陰性と呼び、
予測が陽性であるか、陰性であるか、
実際に陽性であるか、陰性であるかの組み合わせに対して、
真陽性・偽陽性・真陰性・偽陰性という用語を用います。

|  | 予測が陽性 | 予測が陰性 |
| :---- | :----: | :----: |
| 実際は**陽性** | 真陽性数（TP) | 偽陰性数（FN) |
| 実際は**陰性** | 偽陽性数（FP) | 真陰性数（TN）|


> - 真陽性 (Truly Positive, TP): 予測ラベル・正解ラベルともに陽性 
> - 真陰性 (Truly Negative, TN): 予測ラベル・正解ラベルともに陰性 
> - 偽陽性 (Falsely Positive, FP): 予測ラベルが陽性、正解ラベルが陰性 
> - 偽陰性 (Falsely Negative, FN): 予測ラベルが陰性、正解ラベルが陽性

混同行列から種々の指標が導かれます。


- $\displaystyle
    \textbf{正解率（Accuracy）} = \frac{\text{真陽性数} + \text{真陰性数}}{\text{真陽性数} + \text{真陰性数} + \text{偽陽性数} + \text{偽陰性数}}
$

- $\displaystyle
    \textbf{再現率（Recall）} = \frac{\text{真陽性数}}{\text{真陽性数} + \text{偽陰性数}}
$

- $\displaystyle
    \textbf{適合率（Precision）} = \frac{\text{真陽性数}}{\text{真陽性数} + \text{偽陽性数}}
$

- $\displaystyle
    \textbf{F-値（F-score）} = \frac{2 \times \text{真陽性数}}{2\times\text{真陽性数} + \text{偽陽性数} + \text{偽陰性数}}
$

- $\displaystyle
    \textbf{特異度（Specificity）} = \frac{\text{真陰性数}}{\text{真陰性数} + \text{偽陽性数}}
$


> #### 正解率
>全テスト数に対する正答数の割合です。
> #### 再現率
> 検査の陽性者は、更に詳細な検査を受け、その結果罹患が確認されれば、
医療措置が施されることになります。
>真の陽性者うち陽性と予測された者の割合であり、
検査薬の例では、
「**罹患者の何割を検査薬で検出することができるか**」を示す指標となります。
再現率の値が低いと、多くの罹患者を検査で見逃し、
治療の機会を逸してしまう結果になります。
>#### 適合率
>陽性と予想された者のうち真に陽性である者の割合であり、
検査薬の例では、
「**陽性判定者の何割が実際の罹患者か**」を示す指標となります。
陽性率の値が低いことは、検査結果の信頼性が低いことを示し、
その後の詳細な検査のコストが大きくなることを意味します。
> #### F-値
> 検査薬の性能を評価する上で、再現率・適合率はともに等しく重要です。
つまり、よい検査薬は再現率と適合率のバランスがとれている必要がある訳です。
**F値は再現率と適合率のバランスを評価するための指標**であって、
その計算には**調和平均**を用います。
>
> 単純に算術平均を使うと、再現率・適合率の一方だけが小さい状態では、
平均はなお大きな値を示す恐れがあります。
しかしながら、
検査薬では適合率・再現率のいずれか一方だけでも非常に小さいと、
検査薬としては使い物にならないので、F値も0に近くなる必要があるため、
算術平均を使うことは適切でないと考えられるからです。